In [5]:
# 由于下载的文件无法解压 数据处理部分暂时省略

In [71]:
import os
import io
import surprise
import pickle
from surprise import model_selection
import pickle


### 针对音乐预测

In [25]:
# 重建歌单id到歌单名的映射字典
# {'326644112': '不悔梦归处，只怨太匆匆。'...}
file_path = '/Users/JQC/Desktop/recommendation_system_codes/'
id_to_name_dic = pickle.load(open(file_path+'popular_playlist.pkl', 'rb'), encoding='utf8')
# 歌单名称 到歌单id映射 eg:{'不悔梦归处，只怨太匆匆。': '326644112', ...}
name_to_id_dic = {}
for key, value in id_to_name_dic.items():
    name_to_id_dic[value] = key
    

In [26]:
file_path_ = os.path.expanduser(file_path + 'popular_music_suprise_format.txt')
reader = surprise.Reader(line_format='user item rating timestamp', sep=',')
# 读取文件数据
music_data = surprise.Dataset.load_from_file(file_path_, reader=reader)
# 构建数据集
trainset = music_data.build_full_trainset()

In [27]:
trainset

In [30]:
trainset.all_users

<bound method Trainset.all_users of <surprise.trainset.Trainset object at 0x111a404e0>>

In [31]:
# 开始训练模型
#sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = surprise.KNNBaseline()
algo.train(trainset)

/anaconda3/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [51]:
current_playlist = list(name_to_id_dic.keys())[820]
print("歌单名称", current_playlist)

# 根据字典的映射取出这个歌单对应的id
# 68720536
playlist_raw_id = name_to_id_dic[current_playlist]
# 将原始的id转化为数据集中inner_id
playlist_inner_id = algo.trainset.to_inner_uid(playlist_raw_id)
# 推荐前10个
recomment_list = algo.get_neighbors(playlist_inner_id, k=10)
recomment_list = [algo.trainset.to_raw_uid(iid) for iid in recomment_list]
recomment_list = [id_to_name_dic[uid] for uid in recomment_list]
print("和歌单 《", current_playlist, "》 最接近的10个歌单为：\n")

recomment_list

歌单名称 世事无常，唯愿你好
和歌单 《 世事无常，唯愿你好 》 最接近的10个歌单为：



['【华语】暖心物语 纯白思念',
 '暗暗作祟| 不甘朋友不敢恋人',
 '专属你的周杰伦',
 '「华语歌曲」',
 '[小风收集]21世纪年轻人的音乐',
 '十七岁那年，以为能和你永远',
 '热门流行华语歌曲50首',
 '最易上手吉他弹唱超精选',
 '打开任意门，就有对的人',
 '行车路上，一曲长歌']

### 模板之针对用户进行预测 

In [58]:
# 重建歌曲id到歌曲名的映射字典
# '287140': '梦不落\t孙燕姿',
song_id_name_dic = pickle.load(open(file_path + "popular_song.pkl","rb"), encoding='utf8')
# 歌曲到id的映射 '梦不落\t孙燕姿': '287257',
song_name_id_dic = {}
for key, value in song_id_name_dic.items():
    song_name_id_dic[value] = key


In [61]:
#内部编码的4号用户
user_inner_id = 4
# 获取用户的真实评价
user_rating = trainset.ur[user_inner_id]
# 拿到item的id  (478, 1.0),
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'was_impossible': False} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'was_impossible': False} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'was_impossible': False} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'was_impossible': False} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'was_impossible': False} 可以不可以	丁当
user: 4          item: 941        r_ui = 1.00   est = 1.00   {'was_impossible': False} 秋酿	房东的猫
user: 4          item: 616        r_ui = 1.00   est = 1.00   {'was_impossible': False} 退后	周杰伦
user: 4          item: 942        r_ui = 1.00   est = 1.00   {'was_impossible': False} 阴天	莫文蔚
user: 4        

### 2.用矩阵分解进行预测

In [62]:
music_file_path = os.path.expanduser(file_path + './popular_music_suprise_format.txt')
reader = surprise.Rerader(line_format='user item rating timestamp', sep=',')
data = surprise.Dataset.load_from_file(music_file_path, reader=reader)
# 将所有数据全部转为训练数据 
train_data = data.build_full_trainset()

In [65]:
# NMF 矩阵分解 建模 训练数据
algo = surprise.NMF()
algo.fit(trainset)

In [66]:
#内部编码的4号用户
user_inner_id = 4
# 获取用户的真实评价
user_rating = trainset.ur[user_inner_id]
# 拿到item的id  (478, 1.0),
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 4          item: 478        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 梦一场	萧敬腾
user: 4          item: 936        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 干杯	西瓜Kune
user: 4          item: 937        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 小半	陈粒
user: 4          item: 939        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = 1.00   est = 1.00   {'was_impossible': True, 'reason': 'User and item are unkown.'} 可以不可以	丁当
user: 4          item: 941      

### 3.模型存储

In [68]:
surprise.dump.dump('./recommand.model', algo=algo)
algo = surprise.dump.load('./recommand.model')

### 4.不同的推荐系统算法评估

In [78]:
music_file_path = os.path.expanduser(file_path + './popular_music_suprise_format.txt')
reader = surprise.Reader(line_format='user item rating timestamp', sep=',')
# 这个数据有问题 暂且先补考虑 主要理解这个思路
# data = surprise.Dataset.load_from_file(music_file_path, reader=reader)
data = surprise.Dataset.load_builtin('ml-100k')

In [79]:
# NormalPredictor
norm_algo = surprise.NormalPredictor()
model_selection.cross_validate(norm_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.5150  1.5217  1.5234  1.5201  0.0036  
MAE (testset)     1.2137  1.2210  1.2240  1.2196  0.0043  
Fit time          0.12    0.16    0.13    0.14    0.02    
Test time         0.55    0.21    0.27    0.34    0.15    


{'test_rmse': array([1.51502341, 1.52173312, 1.52340444]),
 'test_mae': array([1.21370909, 1.22097947, 1.22399595]),
 'fit_time': (0.11741399765014648, 0.1614069938659668, 0.1290912628173828),
 'test_time': (0.5503711700439453, 0.20740818977355957, 0.2677040100097656)}

In [80]:
# KNNBaseline 比较下面数据 这个比上面的模型要好一些
line_algo = surprise.KNNBaseline()
pref = model_selection.cross_validate(line_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9354  0.9349  0.9366  0.9356  0.0007  
MAE (testset)     0.7371  0.7363  0.7391  0.7375  0.0012  
Fit time          0.50    0.45    0.45    0.46    0.02    
Test time         6.65    6.31    6.29    6.42    0.17    


In [81]:
# 基础版的协同过滤
basic_algo = surprise.KNNBasic()
pref = model_selection.cross_validate(basic_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9871  0.9913  0.9867  0.9884  0.0021  
MAE (testset)     0.7803  0.7846  0.7806  0.7818  0.0019  
Fit time          0.30    0.30    0.28    0.29    0.01    
Test time         5.28    5.00    5.00    5.09    0.13    


In [82]:
# 均值版的协同过滤
mean_algo = surprise.KNNWithMeans()
pref = model_selection.cross_validate(mean_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9536  0.9606  0.9572  0.9571  0.0029  
MAE (testset)     0.7532  0.7557  0.7537  0.7542  0.0011  
Fit time          0.30    0.32    0.39    0.33    0.04    
Test time         5.81    5.43    5.69    5.64    0.16    


In [83]:
# 均值版的协同过滤
zscore_algo = surprise.KNNWithZScore()
pref = model_selection.cross_validate(zscore_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9552  0.9561  0.9583  0.9565  0.0013  
MAE (testset)     0.7486  0.7498  0.7535  0.7506  0.0021  
Fit time          0.38    0.39    0.38    0.39    0.00    
Test time         7.21    6.44    6.29    6.64    0.40    


In [84]:
svd_algo = surprise.SVD()
pref = model_selection.cross_validate(svd_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9437  0.9513  0.9459  0.9470  0.0032  
MAE (testset)     0.7483  0.7482  0.7461  0.7475  0.0010  
Fit time          3.86    3.84    4.26    3.99    0.19    
Test time         0.43    0.27    0.30    0.33    0.07    


In [85]:
svdPP_algo = surprise.SVDpp()
pref = model_selection.cross_validate(svdPP_algo, data, measures=['rmse', 'mae'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9264  0.9276  0.9283  0.9274  0.0008  
MAE (testset)     0.7275  0.7290  0.7305  0.7290  0.0012  
Fit time          128.78  125.28  128.25  127.44  1.54    
Test time         5.98    5.81    5.74    5.85    0.10    
